In [1]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 1.2 MB/s eta 0:00:00a 0:00:01


In [2]:
from kafka import KafkaConsumer, KafkaProducer
import json
import uuid

def connect_kafka_producer(servers):
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=servers, api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [3]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))

In [4]:
import random
import time

def produce_xy(producer, topic_name):
    while True:
        name = f'Mark {random.randint(0,22)}'
        message = json.dumps({"name": name})
        print(name)
        publish_message(producer, topic_name, str(uuid.uuid4()), message)
        time.sleep(1)

In [5]:
def consume_xy(consumer, topic_name):

    for msg in consumer:
        print(msg.key.decode("utf-8"), msg.value)

In [6]:
server1 = 'broker1:9093'
topic1 = "names"

producer = connect_kafka_producer(server1)

consumer = KafkaConsumer(topic1, 
                         auto_offset_reset='earliest',
                         bootstrap_servers=[server1], 
                         api_version=(0, 10), 
                         value_deserializer = json.loads,
                         consumer_timeout_ms=1000)

In [14]:
produce_xy(producer, topic1)

Mark 15
Message published successfully.
Mark 2
Message published successfully.


KeyboardInterrupt: 

In [15]:
consume_xy(consumer, topic1)

e7b73a19-b0c0-44af-8255-995661757591 {'name': 'Mark 15'}
1505ba3f-d1fa-4d41-8610-46d99404fa54 {'name': 'Mark 2'}


In [1]:
from kafka_functions import *
import uuid

In [2]:
publish_message(twitter_producer, topic_name=twitter_topic, key=str(uuid.uuid4()), value="hallelujah")

Using <function <lambda> at 0x7fb252e97640> as value serializer.
Using <function <lambda> at 0x7fb252ea7760> as key serializer.
Message published successfully.


In [3]:
consume_messages(twitter_consumer)

Consumed 5 messages from Kafka Cluster


[ConsumerRecord(topic='twitter', partition=0, offset=0, timestamp=1678352827588, timestamp_type=0, key='92fd878c-54c9-48a3-940e-e9139c03f44f', value='hallelujah', headers=[], checksum=None, serialized_key_size=36, serialized_value_size=25, serialized_header_size=-1),
 ConsumerRecord(topic='twitter', partition=0, offset=1, timestamp=1678352998375, timestamp_type=0, key='bcfda319-3677-4ac9-a69a-3670950426d1', value='hallelujah', headers=[], checksum=None, serialized_key_size=36, serialized_value_size=25, serialized_header_size=-1),
 ConsumerRecord(topic='twitter', partition=0, offset=2, timestamp=1678353082099, timestamp_type=0, key='a8a4e2dc-f129-4fa3-9118-b8a220bb0d1b', value='hallelujah', headers=[], checksum=None, serialized_key_size=36, serialized_value_size=25, serialized_header_size=-1),
 ConsumerRecord(topic='twitter', partition=0, offset=3, timestamp=1678353289387, timestamp_type=0, key='fd455466-c4f1-4a9c-9fb9-fd0dc1eff75c', value='hallelujah', headers=[], checksum=None, serial

In [4]:
publish_message(binance_producer, topic_name=binance_topic, key=str(uuid.uuid4()), value="test")

Using <function <lambda> at 0x7fb252ea7eb0> as value serializer.
Using <function <lambda> at 0x7fb252f18040> as key serializer.
Message published successfully.


In [6]:
consume_messages(binance_consumer)

Consumed 2 messages from Kafka Cluster


[ConsumerRecord(topic='binance-ws', partition=0, offset=0, timestamp=1678353503437, timestamp_type=0, key=b'a41ac0fa-f728-47fb-a274-a651d14d767f', value='test', headers=[], checksum=2094713400, serialized_key_size=36, serialized_value_size=6, serialized_header_size=-1),
 ConsumerRecord(topic='binance-ws', partition=0, offset=1, timestamp=1678353504630, timestamp_type=0, key=b'fb75f945-93c6-4822-b896-8bad87052208', value={'key': 'value'}, headers=[], checksum=3716824245, serialized_key_size=36, serialized_value_size=16, serialized_header_size=-1)]